# Training gemma

In [1]:
import torch
import time
import pandas as pd
from datasets import Dataset


from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training, 
    TaskType
)
from trl import SFTTrainer
from dataclasses import dataclass, field
from typing import List, Optional

In [2]:
@dataclass
class FinetuneConfig:
    # Model gốc của Google, không dùng hàng Unsloth nữa
    model_name: str = "google/gemma-2-2b" 
    max_seq_length: int = 2048
    
    # LoRA Params
    lora_rank: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    # Native PEFT cần list target modules chính xác
    target_modules: List[str] = field(default_factory=lambda: [
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ])
    
    # Train Params
    learning_rate: float = 2e-4
    batch_size: int = 2
    gradient_accumulation_steps: int = 4
    max_steps: int = 60
    warmup_steps: int = 5
    logging_steps: int = 1
    seed: int = 3407
    
    output_dir: str = "models/gemma_native_finetuned"
    dataset_path: str = "../data/question_answer/datasetdone.xlsx"

In [3]:
config = FinetuneConfig()

## 1. Model Loading

In [4]:
import torch
from peft import LoraConfig, TaskType
from transformers import BitsAndBytesConfig

### 1.1. LoRA configuration

In [5]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

### 1.2. Quantization configuration

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [7]:
print(f"📂 Loading Model: {config.model_name}")
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16
)
model = prepare_model_for_kbit_training(model)

📂 Loading Model: google/gemma-2-2b


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
print(f"📂 Loading Tokenizer: {config.model_name}")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

📂 Loading Tokenizer: google/gemma-2-2b


In [9]:
print("🛠️ Attaching LoRA adapters...")
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

🛠️ Attaching LoRA adapters...
trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


### 1.3. Load model and QLoRA attachment

In [12]:
import torch
from trl import SFTTrainer, SFTConfig

def run_finetuning(
    config: FinetuneConfig,
    model,
    tokenizer,
):
    # --- 0. CHECK CUDA & GPU INFO ---
    print("\n" + "="*40)
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"🔥 GPU DETECTED: {gpu_name} ({vram_gb:.2f} GB VRAM)")
        print(f"   CUDA Version: {torch.version.cuda}")
    else:
        print("⚠️ WARNING: KHÔNG TÌM THẤY GPU! Đang chạy bằng CPU (sẽ rất chậm).")
    print("="*40 + "\n")

    versioning = time.strftime("%Y%m%d-%H%M%S")
    run_output_dir = f"{config.output_dir}/{versioning}"
    print(f"🚀 Starting run: {versioning}")

    # --- 1. Load Data & Format ---
    print(f"📚 Loading Data from: {config.dataset_path}")
    try:
        df = pd.read_excel(config.dataset_path)
        if 'Column2' not in df.columns or 'Column3' not in df.columns:
             df.columns = ['Column2', 'Column3'] + list(df.columns[2:])
    except Exception as e:
        print(f"❌ Lỗi đọc Excel: {e}")
        return

    dataset = Dataset.from_pandas(df)
    def formatting_prompts_func(examples):
        texts = [f"<start_of_turn>user\n{q}<end_of_turn>\n<start_of_turn>model\n{a}<end_of_turn>" 
                 for q, a in zip(examples["Column2"], examples["Column3"])]
        return { "text" : texts }
    dataset = dataset.map(formatting_prompts_func, batched=True)

    # --- 2. Config & Trainer Setup ---
    print("⚙️ Setting up Trainer...")
    
    sft_config = SFTConfig(
        output_dir=run_output_dir,
        
        # === DATASET & LENGTH (Quan trọng) ===
        dataset_text_field="text",
        packing=False,
        
        # === GPU & PERFORMANCE ===
        per_device_train_batch_size=config.batch_size,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        warmup_steps=config.warmup_steps,
        max_steps=config.max_steps,
        learning_rate=config.learning_rate,
        fp16=True,                # Bắt buộc True để chạy tốt trên GPU T4/Consumer
        # bf16=True,              # Bật cái này nều dùng RTX 3090/4090 hoặc A100 (tắt fp16 đi)
        optim="paged_adamw_8bit", # Optimizer tiết kiệm VRAM
        
        # === TIẾN TRÌNH & LOGGING (Hiển thị thanh chạy) ===
        logging_steps=1,          # In log sau MỖI bước (để thấy nó chạy ngay)
        logging_first_step=True,  # In ngay bước đầu tiên
        report_to="none",         # In ra màn hình console (không gửi lên wandb)
        disable_tqdm=False,       # Đảm bảo thanh loading bar hiện lên
        save_strategy="no",       # Không save checkpoint rác giữa chừng cho nhẹ ổ cứng
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        processing_class=tokenizer,
        args=sft_config,
    )

    # --- 3. RUN ---
    print("\n🔥 TRAINING STARTED... (Nếu thấy Loss giảm là ngon)")
    try:
        trainer.train()
    except Exception as e:
        print(f"\n❌ Lỗi khi train: {e}")
        print("💡 Gợi ý: Nếu lỗi CUDA OOM (Out of Memory), hãy giảm batch_size hoặc max_seq_length.")
        return

    # --- 4. Save ---
    print(f"\n💾 Saving adapter to {run_output_dir}...")
    trainer.save_model(run_output_dir)
    tokenizer.save_pretrained(run_output_dir)
    print("✅ DONE! TRAINING COMPLETED.")

In [13]:
if __name__ == "__main__":
    # Khởi tạo config
    conf = FinetuneConfig()
    # Chạy
    run_finetuning(
        conf,
        model=model,
        tokenizer=tokenizer
    )


🔥 GPU DETECTED: NVIDIA GeForce RTX 3050 6GB Laptop GPU (6.44 GB VRAM)
   CUDA Version: 12.1

🚀 Starting run: 20251129-180733
📚 Loading Data from: ../data/question_answer/datasetdone.xlsx


Map:   0%|          | 0/275 [00:00<?, ? examples/s]

⚙️ Setting up Trainer...


Adding EOS to train dataset:   0%|          | 0/275 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/275 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/275 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 1}.



🔥 TRAINING STARTED... (Nếu thấy Loss giảm là ngon)


Step,Training Loss
1,2.768200
2,3.346000
3,2.114700
4,2.354400
5,2.328200
6,2.311100
7,2.192400
8,2.019900
9,1.706700
10,1.991200



💾 Saving adapter to models/gemma_native_finetuned/20251129-180733...
✅ DONE! TRAINING COMPLETED.
